## 目标
使用预训练的大语言模型（LLM），通过Prompt方式，对输入的新闻资讯进行解读，评估并返回利好程度。
可选的LLM包括：  
· Deepseek 


## 方案
1. DS API默认不记住上下文，每轮对话都需要重新发送设定信息。不支持指定conversation_id来实现多轮对话连贯性。
2. 即使一次对话中给了多条user身份的content，DS API也仅对第一条返回response。
3. 如果将多条问题通过json格式一次性塞进单条content中，并要求同样在返回的单条response中用json包括多条回答，也可以做到，但是解析起来比较麻烦，节约不了太多Token。
4. 最终设计方案：逐条新闻解读，每轮对话用system身份设定背景条件，单条新闻附加到user身份的content。

In [ ]:
import openai
import pandas as pd
import json
import requests
import re
import os

from dotenv import dotenv_values, load_dotenv
load_dotenv('.env') # 从配置文件中读取密钥，实现配置和代码分离。参考： https://juejin.cn/post/7099283807953977358


In [51]:
# 读取财联社-电报
stock_cls_telegram_df = pd.read_csv('data/stock_cls_telegram.csv')
# 数据清洗：去除NaN值，填充为空字符串
stock_cls_telegram_df = stock_cls_telegram_df.fillna('') 
# '利好评估'列：评估本条新闻对股票的利好程度，单值，从以下选项中选取一个：强烈利好、利好、中性、利空、强烈利空、无法判断
# '新闻解读'列：对本条新闻使用自然语言进行解读，给出思考过程
# '评估状态'列：用于存储评估状态，从以下选项中选取一个：已评估 ，''-未评估，无法评估
# 如果不存在'利好评估'、‘新闻解读’、‘已评估’列，则增加
if '利好评估' not in stock_cls_telegram_df.columns:
    stock_cls_telegram_df['利好评估'] = ''
if '新闻解读' not in stock_cls_telegram_df.columns:
    stock_cls_telegram_df['新闻解读'] = ''
if '评估状态' not in stock_cls_telegram_df.columns:
    stock_cls_telegram_df['评估状态'] = ''
if '金价-评估' not in stock_cls_telegram_df.columns:
    stock_cls_telegram_df['金价-评估'] = ''
if '金价-新闻解读' not in stock_cls_telegram_df.columns:
    stock_cls_telegram_df['金价-新闻解读'] = ''
if '金价-评估状态' not in stock_cls_telegram_df.columns:
    stock_cls_telegram_df['金价-评估状态'] = ''


In [52]:
# 定义一个用户计算token数量的函数，防止超过模型的token数量限制。
# 需要注意的是，此处仅计算输入的Token数量，不包括模型自动生成的Token数量。实际的限制中，还需要加上模型自动生成的Token数量。

def num_tokens_from_messages(messages, model="deepseek-r1"):
    """Return the number of tokens used by a list of messages."""
    import transformers
    chat_tokenizer_dir = "./"
    tokenizer = transformers.AutoTokenizer.from_pretrained( 
        chat_tokenizer_dir, trust_remote_code=True
        )
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")

    num_tokens = 0
    for message in messages:
        num_tokens += tokenizer.encode(message)
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += 1
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [ ]:
# 功能：使用DS GPT对新闻进行解读
import os
import requests
import re
url = "https://wcode.net/api/gpt/v1/chat/completions"

api2d_Authorization = os.environ.get("DS_key")
headers = {
  'Content-Type': 'application/json',
  'Authorization': api2d_Authorization
}

# messages：对话列表，包含系统提示和用户输入
messages = []

# system_prompt：比较复杂，从本地文件读取
system_prompt = open('system_prompt.md', 'r').read()
messages = [{"role": "system", "content": system_prompt}]

# 构建token数量限制的字典。tokens限制：https://platform.openai.com/docs/models/gpt-3-5
# token_limit = {"gpt-3.5-turbo": 4096,"gpt-3.5-turbo-0613": 4096,  "gpt-3.5-turbo-16k": 16384,"gpt-3.5-turbo-16k-0613": 16384, "gpt-4": 8192,"gpt-4-0613": 8192, "gpt-4-32k": 32768,  "gpt-4-32k-0613": 32768}

# 定义新闻评估函数，功能包括：计算messages的token数量、根据messages批量生成新闻解读
# 入参包括：模型消息 messages，模型名称 model
def stock_eval(messages, model):
    
    # # 计算messages的token数量。
    # messages_tokens=num_tokens_from_messages(messages, model)
    # print(f"messages_tokens:{messages_tokens}")

    # 生成一次新闻解读
    # response=openai.ChatCompletion.create(  # OpenAI接口国内连不通
    #     model=model, 
    #     messages=messages,
    #     max_tokens=token_limit[model] - messages_tokens-100, # 此处定义可返回的最大tokens数量。限制条件：messages的tokens + max_tokens返回的最大tokens <= token_limit 总tokens限制。留100个tokens用于宽限。
    #     temperature=0 # between 0 and 2. Lower values like 0.2 will make it more focused and deterministic.
    # )

    # 使用api2d.net代理
    url = "https://wcode.net/api/gpt/v1/chat/completions"
    # messages_tokens=num_tokens_from_messages(messages, model)
    payload = json.dumps({
    "model": model,
    "messages":messages,
    "safe_mode": False,
    # "max_tokens": token_limit[model] - messages_tokens -100, # 此处定义可返回的最大tokens数量。限制条件：messages的tokens + max_tokens返回的最大tokens <= token_limit 总tokens限制。留100个tokens用于宽限。
    "temperature":0 # between 0 and 2. Lower values like 0.2 will make it more focused and deterministic.
    })
    headers = {
    'Authorization': api2d_Authorization,
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)
    response_dict = response.json()  # 将响应内容转换为字典

    content=response_dict['choices'][0]['message']['content'] 
    print(f"Response content: {content}")
    try:
        # eval = re.findall(r'eval:\s*(.*?)\s*(?:\\n|$)', content) # 解析出利好评估
        # cot = re.findall(r'COT:\s*(.*?)\s*(?:\\n|$)', content) # 解析出新闻解读
        eval = re.findall(r'eval:\s*(.*?)\s*(?:\n|$)', content, re.DOTALL)[0]
        cot = re.findall(r'COT:\s*(.*?)\s*(?:\n|$)', content, re.DOTALL)[0]
        status = '已评估'
    except:
        eval = ''
        cot = ''
        status = ''
    return {"eval": eval, "cot": cot, "status": status}

# 构建user_prompt。
model="deepseek-r1" # 调用模型名称
# print(f"system_prompt tokens: {num_tokens_from_messages(messages, model)}")

for index, row in stock_cls_telegram_df.iterrows():
    if row['匹配证券代码'] == '' : # 如果 ‘匹配证券代码’ 为空，则 '评估状态' 列填充为 '无法评估'
        stock_cls_telegram_df.loc[index, '评估状态'] = '无法评估'
        continue # 跳出本次循环，继续下一次循环
    if row['评估状态'] == '' :  # 需要评估的新闻，仅包含 '评估状态'为空 的行
        # 通过本条新闻，构建单条user_prompt
        # user_prompt：读取stock_cls_telegram_eval_df中的'提取证券名称'、‘内容’列，构建user_prompt
        user_prompt = "stock"+row['提取证券名称']+"\n"+"content: "+row['内容']
        print(user_prompt)
        user_prompt = json.dumps(user_prompt,ensure_ascii=False) # 将字典转换为json格式的文本类型，否则会报错。ensure_ascii=False，保证中文不会被转换为ascii码。
        messages.append({"role": "user", "content": user_prompt})
        result = stock_eval(messages, model) # 调用新闻评估函数
        messages = [{"role": "system", "content": system_prompt}] # 清空并重新构建messages
        # 保存结果
        stock_cls_telegram_df.loc[index, '利好评估'] = result["eval"]
        stock_cls_telegram_df.loc[index, '新闻解读'] = result["cot"]
        stock_cls_telegram_df.loc[index, '评估状态'] = result["status"]
        stock_cls_telegram_df.to_csv('data/stock_cls_telegram.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 功能：使用DS GPT对新闻进行解读
import os
import requests
import re
url = "https://wcode.net/api/gpt/v1/chat/completions"

api2d_Authorization = os.environ.get("DS_key")
headers = {
  'Content-Type': 'application/json',
  'Authorization': api2d_Authorization
}

# messages：对话列表，包含系统提示和用户输入
messages = []

# system_prompt：比较复杂，从本地文件读取
system_prompt = open('system_prompt-gold.md', 'r').read()
messages = [{"role": "system", "content": system_prompt}]

# 构建token数量限制的字典。tokens限制：https://platform.openai.com/docs/models/gpt-3-5
# token_limit = {"gpt-3.5-turbo": 4096,"gpt-3.5-turbo-0613": 4096,  "gpt-3.5-turbo-16k": 16384,"gpt-3.5-turbo-16k-0613": 16384, "gpt-4": 8192,"gpt-4-0613": 8192, "gpt-4-32k": 32768,  "gpt-4-32k-0613": 32768}

# 定义新闻评估函数，功能包括：计算messages的token数量、根据messages批量生成新闻解读
# 入参包括：模型消息 messages，模型名称 model
def stock_eval(messages, model):
    
    # # 计算messages的token数量。
    # messages_tokens=num_tokens_from_messages(messages, model)
    # print(f"messages_tokens:{messages_tokens}")

    # 生成一次新闻解读
    # response=openai.ChatCompletion.create(  # OpenAI接口国内连不通
    #     model=model, 
    #     messages=messages,
    #     max_tokens=token_limit[model] - messages_tokens-100, # 此处定义可返回的最大tokens数量。限制条件：messages的tokens + max_tokens返回的最大tokens <= token_limit 总tokens限制。留100个tokens用于宽限。
    #     temperature=0 # between 0 and 2. Lower values like 0.2 will make it more focused and deterministic.
    # )

    # 使用api2d.net代理
    url = "https://wcode.net/api/gpt/v1/chat/completions"
    # messages_tokens=num_tokens_from_messages(messages, model)
    payload = json.dumps({
    "model": model,
    "messages":messages,
    "safe_mode": False,
    # "max_tokens": token_limit[model] - messages_tokens -100, # 此处定义可返回的最大tokens数量。限制条件：messages的tokens + max_tokens返回的最大tokens <= token_limit 总tokens限制。留100个tokens用于宽限。
    "temperature":0 # between 0 and 2. Lower values like 0.2 will make it more focused and deterministic.
    })
    headers = {
    'Authorization': api2d_Authorization,
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)
    response_dict = response.json()  # 将响应内容转换为字典

    content=response_dict['choices'][0]['message']['content'] 
    print(f"Response content: {content}")
    try:
        # eval = re.findall(r'eval:\s*(.*?)\s*(?:\\n|$)', content) # 解析出利好评估
        # cot = re.findall(r'COT:\s*(.*?)\s*(?:\\n|$)', content) # 解析出新闻解读
        eval = re.findall(r'eval:\s*(.*?)\s*(?:\n|$)', content, re.DOTALL)[0]
        cot = re.findall(r'COT:\s*(.*?)\s*(?:\n|$)', content, re.DOTALL)[0]
        status = '已评估'
    except:
        eval = ''
        cot = ''
        status = ''
    return {"eval": eval, "cot": cot, "status": status}

# 构建user_prompt。
model="deepseek-r1" # 调用模型名称
# print(f"system_prompt tokens: {num_tokens_from_messages(messages, model)}")

for index, row in stock_cls_telegram_df.iterrows():

    if row['金价-评估状态'] == '' :  # 需要评估的新闻，仅包含 '评估状态'为空 的行
        # 通过本条新闻，构建单条user_prompt
        # user_prompt：读取stock_cls_telegram_eval_df中的'提取证券名称'、‘内容’列，构建user_prompt
        user_prompt = "stock"+row['提取证券名称']+"\n"+"content: "+row['内容']
        print(user_prompt)
        user_prompt = json.dumps(user_prompt,ensure_ascii=False) # 将字典转换为json格式的文本类型，否则会报错。ensure_ascii=False，保证中文不会被转换为ascii码。
        messages.append({"role": "user", "content": user_prompt})
        result = stock_eval(messages, model) # 调用新闻评估函数
        messages = [{"role": "system", "content": system_prompt}] # 清空并重新构建messages
        # 保存结果
        stock_cls_telegram_df.loc[index, '金价-评估'] = result["eval"]
        stock_cls_telegram_df.loc[index, '金价-新闻解读'] = result["cot"]
        stock_cls_telegram_df.loc[index, '金价-评估状态'] = result["status"]
        stock_cls_telegram_df.to_csv('data/stock_cls_telegram.csv', index=False, encoding='utf-8-sig')